In [1]:
pip install mpi4py

Note: you may need to restart the kernel to use updated packages.


In [2]:
from mpi4py import MPI
import numpy as np
import time

In [3]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [4]:
def f(x):
    return x**2

In [29]:
def trap(a, b, n):
    h = (b - a) / n
    local_n = n // size
    local_a = a + rank * local_n * h
    local_b = local_a + local_n * h
    local_integral = (np.exp(a) + np.exp(b)) / 2.0

    for i in range(1, local_n):
        x = local_a + i * h
        local_integral += f(x)

    local_integral *= h
    return local_integral

In [30]:
if rank == 0:
    a, b, n = map(int, input("Please enter a, b, n: ").split())
    start_time = time.time()
else:
    a, b, n = None, None, None

a = comm.bcast(a, root=0)
b = comm.bcast(b, root=0)
n = comm.bcast(n, root=0)

Please enter a, b, n: 0 3 1024


In [31]:
local_integral = trap(a, b, n)
local_integral_array = np.array(local_integral) 
total_integral = np.array(0.0)
comm.Reduce(local_integral_array, total_integral, op=MPI.SUM, root=0)

In [34]:
if rank == 0:
    end_time = time.time()
    execution_time = end_time - start_time
    print(f" With n = {int(n)} trapezoids,\n my estimate of the integral from {a:.6f} to {b:.6f} = {total_integral:.6f},\n execution time is {execution_time:.6f}")

 With n = 1024 trapezoids,
 my estimate of the integral from 0.000000 to 3.000000 = 9.017708,
 execution time is 19.279995


In [ ]:
'''
Student name: Khalid Nimri
Student ID: 2140145

'''